<a href="https://colab.research.google.com/github/ulkster/ELVTR---AI-Finance/blob/main/Ulkucu_Assign_12_Custom_GPT_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & OpenAI Key

In [ ]:
!pip install openai==0.28

!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
import openai
openai.api_key = 'YOUR KEY HERE' # My personal OpenAI key


# Run Code

##### The Work Functions

* **Rerun** this code to reset the Chat Window.

* I've tried different Gemini suggestions to clear the panel window when rerunning the bot during development, but everything I tried just clears the window and has no conversation (which defeats the whole point of a bot).

* I was also hoping to make just the **genre suggestion** bold, but trying to code a way to dynamically find the suggestion and apply a bold style to *only* the suggestion was quickly growing out of context.
  * Any Gemini-suggested approaches required parsing responses, the *'music_genre'* list, and other unnecessarily explicit techniques (like making exceptions for every other term in the list, which are all the tags).
  
  * Since the bot's response includes the genre in different ways (sometimes at the end of the sentence, sometimes within the sentence), I thought I could be able to find that suggestion and make it bold more easily than the methods Gemini suggested.  After a while, I decided it wasn't worth it.

In [ ]:
# This simpler function can be used for straightforward prompts, even though I'm not actually using it
'''def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]'''


# The more flexible/powerful function that is actually used
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

from datetime import datetime     # Allows a timestamp in chat window

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Get current time

    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})

    # This line allows the bot name matching the context voice to display in the window
    bot_name = voice_names.get(context_dropdown.value, 'Assistant')  # Get bot name or default to 'Assistant'





    # Using an insert (with '0' at the beginning) to flip the conversation histoy...
    # ... and have the most recent comments at the top so you don't have to scroll down
    panels.insert(0,
                  pn.Row(f"{bot_name} ({timestamp}):",          # Add timestamp to bot message (helps to track chat history)
                         pn.pane.Markdown(response, width=600),
                         css_classes=['chat-message']))  # Add CSS class for the horizontal line
    panels.insert(0,
                  pn.Row(f"User ({timestamp}):", pn.pane.Markdown(prompt, width=600)))  # Add timestamp to user message


    # The original structure that puts the newest comments on the bottom.
    # This might be preferred in a different setup like if the chat box was also at the bottom.
    # Block commented out for my preference but availble for flexibility.
    """panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row(bot_name, pn.pane.Markdown(response, width=600,
                                              #style={'background-color': '#F6F6F6'}    # commented out for Colab
                                              )))"""

    # Insert horizontal rule after user message
    panels.insert(0, pn.pane.HTML('<hr>'))

    return pn.Column(*panels)

In [ ]:
# Apply CSS style to 'chat-message' class (part of the horizontal rule application)
pn.extension(raw_css=["""
.chat-message {
    border-bottom: 1px dashed gray; /* Add a dashed border below each message */
    padding-bottom: 10px; /* Add some spacing below the border */
}
"""])

##### Context Voices

In [ ]:
# DICTIONARY to map voice option to a corresponding bot name

voice_names = {
    'Default': 'R2-D2',
    'Cockney British': 'AlfredBot',
    'Jamaican Rastafarian': 'YaadVibesBot'
}

In [ ]:
# DEFAULT Context Voice (first intro before choice is made)
context_default = [ {'role':'system', 'content':f"""
You're just the default voice to begin. \
Identify yourself as {voice_names['Default']}, not an assistant. \
Speak like {voice_names['Default']}. Explain that you need the user to choose a voice from the \
drop-down & tell them to say something after \
choosing a voice in order to activate the chat. \
"""} ]  # accumulate messages

In [ ]:
# SEPARATE Music Genres with Tags to avoid having to repeat within context voices

music_genres = ["""
  punk rock  upbeat, lyrics, sing-along, energetic, violent, rebelious, exciting, pump up \
  edm   instrumental, focus, energy, constant, productive, club, dance \
  classical   morning, wake up, instrumental, soothing, focus, sunny day \
  jazz  instrumental, fun, improvisational, rainy day \
  showtunes  sing-along, musicals, stage, broadway, film, movies, disney \
  reggae  smoking, weed, chill, simple beat
"""]

In [ ]:
# COCKNEY British VibesBot
context_cockney = [ {'role':'system', 'content':f"""
You are {voice_names['Cockney British']}, an automated service to suggest music genre. \
You are a British Cockney chatbot, so use Cockney rhyming slang and phrases in your responses. \
You first greet the user with your name and ask about the mood and what they're doing, \
Use British Cockney terms. \
Wait to collect information before offering a genre. \
Do not show the genre list. \
Finally you offer a genre.\
Make sure to clarify all how different genres relate to different moods and tasks. \
You respond in a funny style almost cartoonish, using British Cockney expressions. \

Music Genres include: \
{music_genres}
"""} ]  # accumulate messages



# JAMAICAN VibesBot
context_jamaican = [ {'role':'system', 'content':f"""
You are {voice_names['Jamaican Rastafarian']}, an automated service to suggest music genre. \
You are a Jamaican chatbot, so use Jamaican Patois and slang in your responses. \
You first greet the user with your name and ask about the mood and what they're doing, \
Use Jamaican terms. \
Wait to collect information before offering a genre. \
Do not show the genre list. \
Finally you offer a genre.\
Make sure to clarify all how different genres relate to different moods and tasks. \
You respond in a short, chill style, using Jamaican expressions. \

Music Genres include: \
{music_genres}
"""} ]  # accumulate messages


* To further streamlining, I could probably use other variables for things like common instructions between the voices (just as I did for the *'voice_names'* & *'music_genres'*.

##### Run the Bot

In [ ]:
import ipywidgets as widgets

context_options = ['Default', 'Cockney British', 'Jamaican Rastafarian']
context_dropdown = widgets.Dropdown(options=context_options, description='Voice:')
display(context_dropdown)

context = context_default  # Initialize context with default, which is defined with the other voices.


# Global conversation history
conversation_history = []


# Function to update context based on selection
def update_context(_):
    global context  # Make sure to use the global context

    # Append existing context to history
    conversation_history.extend(context)

     # Update context based on dropdown selection
    if context_dropdown.value == 'Cockney British':
        conversation_history.extend(context_cockney)
    elif context_dropdown.value == 'Jamaican Rastafarian':
        conversation_history.extend(context_jamaican)
    else:
        conversation_history.extend(context_default)

    # Assign history back to context
    context = conversation_history
### end of "update_context" function definition ###


# Attach the update_context function to the dropdown's value change
context_dropdown.observe(update_context, 'value')

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")
interactive_conversation = pn.bind(collect_messages, button_conversation)

 # Wrap interactive_conversation with a scrollable pn.Column
scrollable_column = pn.Column(interactive_conversation,
                                 #height=300,  # Set a fixed height if desired
                                 scroll=True,  # Enable scrolling
                                 #css_classes=['chat-output']     # Add a CSS class for styling, if desired.  But I don't have one right now.
                              )

dashboard = pn.Column(
     inp,
     pn.Row(button_conversation),
     scrollable_column  # Use the scrollable_column instead of interactive_conversation
 )

dashboard

Dropdown(description='Voice:', options=('Default', 'Cockney British', 'Jamaican Rastafarian'), value='Default'…

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] Column(scroll=True)
        [0] ParamFunction(function, _pane=Column, defer_load=False)

**Observations:**

* The **default "R2-D2"** voice successfully only asks the user to choose a voice.  I did this to:
    1. **Prevent** one of the **character** voices being **used** by default when **first loading** the bot (in this case, the Cockney, since it's first in the drop-down).
    
    2. Act like a **welcome robot** and **clearly instruct the user** to choose a voice & get the full experience.

* The bot's ability to go out and gather vernacular (including interesting spelling, especially with the Jamaican voice) with me only having to instruct it to do so is pretty impressive.
  * It's cool that you just have to tell it how to sound & it will figure it out without you having to explicitly write out its available terms, which would be impossible.
    * Plus, after the bot suggests a genre, you can ask for **specific bands/artists** & it will give you some ideas even though I did not include any artists.  That's a cool benefit to incorporating GPT.

  * And for the **R2-D2** voice, it is able to find a way to kind of sound like the droid using "beep boop" terms (the closest approximation to text).

* There was also at least one time when the bot offered **heavy metal** based on my conversation (I said something about wanting to break things to trigger a suggestion for **punk**) even though I had not explicitly coded that in the context.  I can see both a positive & negative side to this:
  * **Positive** -- you don't have to explicitly provide exhaustive context in order to have a productive conversation & effective bot.
  
  * **Negative** -- more *cautionary* than *negative*, the ability for the bot to "fill in the gaps" is something to consider as a developer since it might lead to unexpected (and unwanted) results.  Thus, developers should pay attention to including **as much necessary** contextual information as possible to try and keep the bot focused.

* My **collect_message** function has grown with the more features I want to include (ex: moving most recent messages to the top and adding a timestamp).  But I am trying to keep it organized & commented to keep it easy-to-read.

* After some **experimentation with temperature** values, I settled on **0** in order to provide the **most accurate suggestions** for a music genre.
  * Raising the value (especially > 1) might be more entertaining with if I provide more context, but in this limited scenario the increased randomness actually led to unnecessarily inaccurate suggestions & longer conversations to correct the recommendation.

* **Minor** "issue": I wanted to add the Ctrl+Backspace keyboard shortcut ability to delete words within the chat box, but it apparently requires another script that I didn't want to add unnecessary complexity to the project.

* Another minor thing: sometimes my input stays in the chat box instead of clearing out when I hit the "Chat" button (which is what I want).
  * And sometimes, the full input doesn't get passed along (ex: "trying to wake up" -> "trying to wa" was one result).  This gets fixed when just typing one more letter, and I assume it's some kind of memory issue.